<a href="https://colab.research.google.com/github/smavianeelam/brain-tumor-detection-ml/blob/main/main_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
 Author: Smavia Neelam
# This notebook is safe and was created for academic purposes.
# It contains a CNN model for brain tumor detection.
# No external drive access or hidden data requests are made.

In [ ]:
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download mohammadhossein77/brain-tumors-dataset
!kaggle datasets download gazu468/cifar10-classification-image

In [ ]:
!unzip 'brain-tumors-dataset.zip'
!unzip 'cifar10-classification-image.zip'

In [ ]:
import os
import pandas as pd
from random import sample

In [ ]:
def get_image_paths(directory):
    image_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(os.path.join(root, file))
    return image_paths

In [ ]:
cifar10_dir = '/content/cifar10/test'
cifar_classes = ['airplane', 'automobile', 'bird','cat','deer','dog','frog','horse','ship','truck']

In [ ]:
image_paths_cifar = []
for cifar_class in cifar_classes:
    class_dir = os.path.join(cifar10_dir, cifar_class) #(/content/cifar10/test/airplane) (/content/cifar10/test/bird) (/content/cifar10/test/cat) (/content/cifar10/test/deer) (/content/cifar10/test/dog) (/content/cifar10/test/frog) (/content/cifar10/test/horse) (/content/cifar10/test/ship) (/content/cifar10/test/truck)
    class_images = get_image_paths(class_dir)


    selected_images = sample(class_images, 700)
    image_paths_cifar.extend([(img, 0) for img in selected_images])

In [ ]:
brain_mri_dir = '/content/Data/Normal'

In [ ]:
brain_mri_images = get_image_paths(brain_mri_dir)
image_paths_brain = [(img, 1) for img in brain_mri_images]

In [ ]:
combined_data = image_paths_cifar + image_paths_brain #"A","B" = "AB"

In [ ]:
df = pd.DataFrame(combined_data, columns=['image_path', 'brain'])

In [ ]:
df

In [ ]:
df['brain'].value_counts().plot(kind='barh')
print(df['brain'].value_counts())

In [ ]:
b={
    0: 'non_brain',
    1: 'brain',
}

In [ ]:
df['class'] = df['brain'].map(b.get)

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split
# test data set, train data set, validation data set
#10%, 80%, 10%
#15% 70%, 15%

In [ ]:
train_int , val = train_test_split(df, test_size= 0.15, stratify=df['brain'])
train, test = train_test_split(train_int, test_size= 0.15 / (1 - 0.15), stratify= train_int['brain'])

In [ ]:
print(train.shape)
print(test.shape)
print(val.shape)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
path = '/content'

In [ ]:
datagen= ImageDataGenerator(

        horizontal_flip=True,
        rescale=1/255,

)

datagen_2 = ImageDataGenerator(
    rescale=1/255
)


train_data_gen = datagen.flow_from_dataframe(
    dataframe= train,
    directory= path,
    x_col="image_path",
    y_col="class",
    target_size=(32,32),
    batch_size=32,
    class_mode="binary"
)

test_data_gen = datagen_2.flow_from_dataframe(
    dataframe= test,
    directory= path,
    x_col="image_path",
    y_col="class",
    target_size=(32,32),
    batch_size=32,
    class_mode="binary"
)

val_data_gen = datagen_2.flow_from_dataframe(
    dataframe= val,
    directory= path,
    x_col="image_path",
    y_col="class",
    target_size=(32,32),
    batch_size=32,
    class_mode="binary"
)

In [ ]:
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing import image

num=5

train_data = val

sample_indices= train_data.sample(num).index

plt.figure(figsize=(12,6))
for i, idx in enumerate(sample_indices):
  img_path= train_data.loc[idx,"image_path"]
  img= image.load_img(img_path,target_size=(150,150))
  plt.subplot(1, num, i+1)
  plt.imshow(img)
  plt.title(train_data.loc[idx, "class"])
  plt.axis("off")

plt.show()

In [ ]:
from keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense
from keras.models import Sequential

In [ ]:
my_model = Sequential()


my_model.add(Conv2D(16,(3,3),activation='relu',input_shape=(32,32,3)))
my_model.add(MaxPool2D((2,2)))


my_model.add(Conv2D(32,(3,3),activation='relu'))
my_model.add(MaxPool2D((2,2)))

my_model.add(Conv2D(64,(3,3),activation='relu'))
my_model.add(MaxPool2D((2,2)))

my_model.add(Dropout(0.25))

my_model.add(Flatten())


my_model.add(Dense(64,activation='relu'))
my_model.add(Dense(32,activation='relu'))
my_model.add(Dense(1,activation='sigmoid'))


my_model.summary()



In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

es=EarlyStopping(monitor="val_accuracy",min_delta=0.01,patience=40,verbose=1,mode='auto')
mc=ModelCheckpoint(monitor="val_accuracy",filepath='./bestmodel.keras',verbose=1,save_best_only=True,mode='auto')


cd = [es,mc]

In [ ]:
import tensorflow as tf

my_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
hs=my_model.fit(train_data_gen,epochs=100,verbose=1,steps_per_epoch=20,validation_data=val_data_gen,validation_steps=20,callbacks=cd)

In [ ]:
h = hs.history
h.keys()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(h['loss'],c="red")
plt.plot(h['val_loss'],c="green")
plt.title("Loss vs val_loss")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(h['accuracy'],c="red")
plt.plot(h['val_accuracy'],c="green")
plt.title("Loss vs val_loss")
plt.show()

In [ ]:
model = tf.keras.models.load_model('/content/bestmodel.keras')
acc= model.evaluate(test_data_gen)[1]
acc*100

In [ ]:
model.save('./bestmodel.h5')